In [1]:
from dotenv import load_dotenv
from polygon import RESTClient
from datetime import datetime,timezone
import os

In [2]:
import pandas as pd

In [8]:
import numpy as np

In [3]:

load_dotenv("../guidelight-api/.env")
token = os.getenv("POLYGON_TOKEN")

In [4]:

client = RESTClient(api_key=token)

In [5]:
ticker = "AAPL"

# List Aggregates (Bars)
aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2019-01-01", to="2023-12-30", limit=50000):
    aggs.append(a)

print(len(aggs))



926635


In [6]:
aggs[-1]

Agg(open=192.1505, high=192.1505, low=192.1501, close=192.1501, volume=2147, vwap=192.1512, timestamp=1703897940000, transactions=12, otc=None)

In [13]:
#unix millisecond timestamp to datetime
# from pytz import timezone

# Convert timestamp to datetime in UTC timezone
def ts_to_dt(timestamp):
    return datetime.fromtimestamp(timestamp/1000, tz=timezone.utc)

timestamps = np.vectorize(ts_to_dt)

# Example usage
a = np.array([a.timestamp for a in aggs])
times = timestamps(a)
aapl_aggs = pd.DataFrame({"day": [time.strftime("%Y-%m-%d") for time in times],
                            "timestamp": [time.strftime("%Y-%m-%d, %H:%M") for time in times],
                           "close": [a.close for a in aggs], 
                           "volume": [a.volume for a in aggs],
                           "low": [a.low for a in aggs],
                           "high": [a.high for a in aggs]})



,day,timestamp,close,volume,low,high
0,2019-04-04,"2019-04-04, 08:00",48.9100,2900.0,48.9100,48.9100
1,2019-04-04,"2019-04-04, 08:01",48.9100,8800.0,48.9100,48.9275
2,2019-04-04,"2019-04-04, 08:06",48.9250,400.0,48.9250,48.9250
3,2019-04-04,"2019-04-04, 08:15",48.9175,3292.0,48.9175,48.9175
4,2019-04-04,"2019-04-04, 08:18",48.9125,708.0,48.9125,48.9125
...,...,...,...,...,...,...
926630,2023-12-30,"2023-12-30, 00:55",192.2400,717.0,192.2400,192.2400
926631,2023-12-30,"2023-12-30, 00:56",192.2300,532.0,192.2300,192.2300
926632,2023-12-30,"2023-12-30, 00:57",192.2200,662.0,192.2200,192.2300
926633,2023-12-30,"2023-12-30, 00:58",192.1000,2154.0,192.1000,192.2200


In [55]:
from heston_param import *

In [17]:
aapl_aggs.groupby("day")

In [42]:
daily_aggs =  [aapl_aggs.groupby('day').get_group(x) for x in aapl_aggs['day'].unique()]

In [57]:
hist_voilatilities = torch.empty(len(daily_aggs))
for i, day in enumerate(daily_aggs):
	hist_voilatilities[i] = estimate_historical_volatility(day['close'].values)

# calculate the historical volatility for each day



# historical_volatility
print(hist_voilatilities)

tensor([0.0004, 0.0004, 0.0005,  ..., 0.0003, 0.0003, 0.0002])
